<a href="https://colab.research.google.com/github/macuriels/umass_dacss_datastorytelling/blob/main/international_sanctions_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install pycountry country_converter networkx xlsxwriter pycountry-convert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.0/229.0 kB 9.6 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import spacy
import pycountry
from country_converter import CountryConverter
import networkx as nx
import io
from google.colab import files
import pycountry_convert as pc

In [ ]:
uploaded = files.upload()

In [7]:
df = pd.read_csv(io.BytesIO(uploaded['GSDB_V3.csv']))
df

,case_id,sanctioned_state,sanctioning_state,begin,end,trade,descr_trade,arms,military,financial,travel,other,target_mult,sender_mult,objective,success
0,1,German Democratic Republic,Germany,1949,1973,0,NaN,0,0,0,0,1,0,0,territorial_conflict,success_total
1,2,Pakistan,India,1949,1951,1,"exp_compl, imp_compl",0,0,0,0,0,0,0,policy_change,nego_settlement
2,3,Bulgaria,United States,1950,1966,0,NaN,0,0,0,0,1,0,0,destab_regime,failed
3,4,Bulgaria,United States,1950,1959,0,NaN,0,0,0,1,0,0,0,destab_regime,success_part
4,5,Bulgaria,United States,1950,1963,0,NaN,0,0,1,0,0,0,0,destab_regime,success_part
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1320,1321,Uganda,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,policy_change,ongoing
1321,1322,Ukraine,United Kingdom,2022,2022,0,NaN,0,0,1,0,0,0,0,end_war,ongoing
1322,1323,United Arab Emirates,United States,2022,2022,1,"exp_part, imp_part",0,0,1,0,0,0,0,terrorism,ongoing
1323,1324,United States,China,2022,2022,0,NaN,0,0,0,1,0,0,0,policy_change,ongoing


In [8]:
# Replace specific patterns in 'sanctioned_state' and 'sanctioning_state' columns
replace_dict = {
    'Congo, Democratic Republic of the, Eritrea, Ethiopia (excludes Eritrea), Kenya, Rwanda, Tanzania, Uganda, Zambia': 'Democratic Republic of the Congo, Eritrea, Ethiopia, Kenya, Rwanda, Tanzania, Uganda, Zambia'
    ,'Congo, Democratic Republic of the': 'Democratic Republic of the Congo'
    ,'Korea, North': 'North Korea'
    ,'Korea, South': 'South Korea'
    ,'Yemen, North': 'North Yemen'
    ,'Gambia, The': 'The Gambia'
    ,'Egypt, Arab Rep.': 'Egypt'
    ,'Egypt, Arab Rep., Syria': 'Egypt, Syria'
    ,'Saudi Arabia, United Arab Emirates, Bahrain, Egypt, Arab Rep.': 'Saudi Arabia, United Arab Emirates, Bahrain, Egypt'
    ,'Laos': "Lao People's Democratic Republic"
    ,'Ethiopia (excludes Eritrea)': 'Ethiopia'
    ,'Congo (Brazzaville)': 'Republic of the Congo'
}

df['sanctioned_state'].replace(replace_dict, inplace=True)
df['sanctioning_state'].replace(replace_dict, inplace=True)

In [9]:
# Function to split values in 'sanctioned_state' into separate rows
def split_countries_and_institutions(row):
    countries_institutions = row['sanctioned_state'].split(',')
    if len(countries_institutions) > 1:
        rows = []
        for country_institution in countries_institutions:
            new_row = row.copy()
            new_row['sanctioned_state'] = country_institution
            rows.append(new_row)
        return rows
    return [row]  # Wrap the single row in a list

# Apply the function and create a list of dictionaries
rows_list = []
for index, row in df.iterrows():
    rows_list.extend(split_countries_and_institutions(row))

# Create a new DataFrame
df = pd.DataFrame(rows_list)

# Reset the index of the new DataFrame
df = df.reset_index(drop=True)

In [10]:
# Function to split values in 'sanctioning_state' into separate rows
def split_countries_and_institutions_column2(row):
    countries_institutions = row['sanctioning_state'].split(',')
    if len(countries_institutions) > 1:
        rows = []
        for country_institution in countries_institutions:
            new_row = row.copy()
            new_row['sanctioning_state'] = country_institution
            rows.append(new_row)
        return rows
    return [row]  # Wrap the single row in a list

# Apply the function and create a list of dictionaries
rows_list_column2 = []
for index, row in df.iterrows():
    rows_list_column2.extend(split_countries_and_institutions_column2(row))

# Create a new DataFrame for the second column
df = pd.DataFrame(rows_list_column2)

# Reset the index of the new DataFrame
df = df.reset_index(drop=True)

In [11]:
# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def filter_organizations(row):
    for col in ['sanctioned_state', 'sanctioning_state']:
        # Check if entry is all upper case
        if row[col].isupper():
            return False

        # Check if entry contains a number
        if any(char.isdigit() for char in row[col]):
            return False

        # Check for specific entries to exclude
        if row[col] == "League of Arab States":
            return False

        # Check if entry contains the word "Balkan"
        if "Balkan" in row[col]:
            return False

        # Continue checking for organization entities using spaCy
        doc = nlp(row[col])
        for ent in doc.ents:
            if ent.label_ == 'ORG':
                return False

    return True

# Apply the function to filter out organization rows
df = df[df.apply(filter_organizations, axis=1)]

In [12]:
# Remove leading white spaces from all cells
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [13]:
# Initialize CountryConverter
cc = CountryConverter()

# Function to get 3-digit country code based on country name
def get_country_code(country_name):
    try:
        country_info = pycountry.countries.search_fuzzy(country_name)
        if country_info:
            return country_info[0].alpha_2
        return None
    except LookupError:
        return None

# Add a new column for 2-digit country codes
df['sanctioned_state_cc'] = df['sanctioned_state'].apply(get_country_code)
df['sanctioning_state_cc'] = df['sanctioning_state'].apply(get_country_code)

# Manually assign country code to DRC
df.loc[df['sanctioned_state'] == 'Democratic Republic of the Congo', 'sanctioned_state_cc'] = 'CD'
df.loc[df['sanctioning_state'] == 'Democratic Republic of the Congo', 'sanctioning_state_cc'] = 'CD'

In [14]:
# Function to generate flag URL
def generate_flag_url(country_code):
    base_url = 'https://www.flagsapi.com'
    size = 'shiny/64.png'  # Change the size if needed (e.g., 16.png, 32.png, etc.)
    return f'{base_url}/{country_code}/{size}'

# Add a new column with flag URLs
df['sanctioned_state_flag'] = df['sanctioned_state_cc'].apply(generate_flag_url)
df['sanctioning_state_flag'] = df['sanctioning_state_cc'].apply(generate_flag_url)

In [21]:
# Drop cases where country codes are null (because they are dissolved countries)
df = df.dropna(subset=['sanctioned_state_cc', 'sanctioning_state_cc'])

In [22]:
# Add a new column for continents
df['sanctioned_state_continent'] = df['sanctioned_state_cc'].apply(
    lambda x: pc.country_alpha2_to_continent_code(country_2_code=x))
df['sanctioning_state_continent'] = df['sanctioning_state_cc'].apply(
    lambda x: pc.country_alpha2_to_continent_code(country_2_code=x))

<ipython-input-22-ef85f2a26260>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sanctioned_state_continent'] = df['sanctioned_state_cc'].apply(
<ipython-input-22-ef85f2a26260>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sanctioning_state_continent'] = df['sanctioning_state_cc'].apply(


In [23]:
df

,case_id,sanctioned_state,sanctioning_state,begin,end,trade,descr_trade,arms,military,financial,...,target_mult,sender_mult,objective,success,sanctioned_state_cc,sanctioning_state_cc,sanctioned_state_flag,sanctioning_state_flag,sanctioned_state_continent,sanctioning_state_continent
1,2,Pakistan,India,1949,1951,1,"exp_compl, imp_compl",0,0,0,...,0,0,policy_change,nego_settlement,PK,IN,https://www.flagsapi.com/PK/shiny/64.png,https://www.flagsapi.com/IN/shiny/64.png,AS,AS
2,3,Bulgaria,United States,1950,1966,0,NaN,0,0,0,...,0,0,destab_regime,failed,BG,US,https://www.flagsapi.com/BG/shiny/64.png,https://www.flagsapi.com/US/shiny/64.png,EU,NA
3,4,Bulgaria,United States,1950,1959,0,NaN,0,0,0,...,0,0,destab_regime,success_part,BG,US,https://www.flagsapi.com/BG/shiny/64.png,https://www.flagsapi.com/US/shiny/64.png,EU,NA
4,5,Bulgaria,United States,1950,1963,0,NaN,0,0,1,...,0,0,destab_regime,success_part,BG,US,https://www.flagsapi.com/BG/shiny/64.png,https://www.flagsapi.com/US/shiny/64.png,EU,NA
6,7,China,New Zealand,1950,1956,1,exp_part,0,0,0,...,0,0,destab_regime,failed,CN,NZ,https://www.flagsapi.com/CN/shiny/64.png,https://www.flagsapi.com/NZ/shiny/64.png,AS,OC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1321,Uganda,United States,2022,2022,1,"exp_part, imp_part",0,0,1,...,0,0,policy_change,ongoing,UG,US,https://www.flagsapi.com/UG/shiny/64.png,https://www.flagsapi.com/US/shiny/64.png,AF,NA
1886,1322,Ukraine,United Kingdom,2022,2022,0,NaN,0,0,1,...,0,0,end_war,ongoing,UA,GB,https://www.flagsapi.com/UA/shiny/64.png,https://www.flagsapi.com/GB/shiny/64.png,EU,EU
1887,1323,United Arab Emirates,United States,2022,2022,1,"exp_part, imp_part",0,0,1,...,0,0,terrorism,ongoing,AE,US,https://www.flagsapi.com/AE/shiny/64.png,https://www.flagsapi.com/US/shiny/64.png,AS,NA
1888,1324,United States,China,2022,2022,0,NaN,0,0,0,...,0,0,policy_change,ongoing,US,CN,https://www.flagsapi.com/US/shiny/64.png,https://www.flagsapi.com/CN/shiny/64.png,NA,AS


In [24]:
# EDGES
df_edges = df.groupby(['sanctioning_state', 'sanctioned_state']).size().reset_index(name='count')

In [25]:
df_edges

,sanctioning_state,sanctioned_state,count
0,Afghanistan,Pakistan,1
1,Albania,Afghanistan,4
2,Albania,Belarus,3
3,Albania,Bosnia and Herzegovina,1
4,Albania,Cote d'Ivoire,2
...,...,...,...
721,Venezuela,Honduras,1
722,Venezuela,Peru,2
723,Venezuela,Suriname,1
724,Yemen,Qatar,1


In [26]:
# NODES
# Reshape the DataFrame
df_nodes = pd.DataFrame({
    'country': df['sanctioning_state'].tolist() + df['sanctioned_state'].tolist(),
    'continent': df['sanctioning_state_continent'].tolist() + df['sanctioned_state_continent'].tolist(),
    'flag': df['sanctioning_state_flag'].tolist() + df['sanctioned_state_flag'].tolist()
})

# Group by distinct pairs of countries and flags
df_nodes = df_nodes.drop_duplicates().reset_index(drop=True)

In [27]:
# Create a directed graph from the dataframe
G = nx.from_pandas_edgelist(df, 'sanctioning_state', 'sanctioned_state', create_using=nx.DiGraph())

# Calculate the node degree for each node
node_degrees = dict(G.degree())

# Append the node degrees to the dataframe
df_nodes['node_degree'] = df_nodes['country'].map(node_degrees)

In [28]:
df_nodes

,country,continent,flag,node_degree
0,India,AS,https://www.flagsapi.com/IN/shiny/64.png,14
1,United States,NA,https://www.flagsapi.com/US/shiny/64.png,120
2,New Zealand,OC,https://www.flagsapi.com/NZ/shiny/64.png,7
3,South Africa,AF,https://www.flagsapi.com/ZA/shiny/64.png,15
4,United Kingdom,EU,https://www.flagsapi.com/GB/shiny/64.png,52
...,...,...,...,...
148,South Sudan,AF,https://www.flagsapi.com/SS/shiny/64.png,13
149,Mali,AF,https://www.flagsapi.com/ML/shiny/64.png,5
150,Qatar,AS,https://www.flagsapi.com/QA/shiny/64.png,15
151,Ghana,AF,https://www.flagsapi.com/GH/shiny/64.png,1


In [29]:
# Export full cleaned dataframe
df.to_csv('gsdb_full.csv')

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter("gsdb_network.xlsx", engine="xlsxwriter")

# Write each dataframe to a different worksheet.
df_nodes.to_excel(writer, sheet_name="Nodes")
df_edges.to_excel(writer, sheet_name="Edges")

# Close the Pandas Excel writer and output the Excel file.
writer.close()